SP500公司名稱前處理

In [ ]:
from cleanco import prepare_terms, basename # 處理公司symbol
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from requests.utils import quote
import pandas as pd
import time
import random
import math
import os

In [ ]:
## 根據SP500的 Company name搜集
read = os.path.join(os.getcwd(),'SP500_merge.xlsx')
symbol_data = pd.read_excel(read, index_col = 0 )
symbol = symbol_data['Company name'].to_list()

# 去除常見贅字
corp_name = []
for i, element in enumerate(list(symbol)):
    ele_low = element.lower()
    ele_low = ele_low.replace(" company", '').replace(' corp', '').replace(' inc', '')\
    .replace(" plc", '').replace(' class a', '').replace(' class c', '')\
    .replace(" class b", '').replace(" ltd.", "").replace(".com", "")\
    .replace(" int'l", "").replace(" corporation", "").replace(" group", "")\
    .replace(" svc.gp.", "").replace(" incorporated", "")
    corp_name.append(ele_low)

# 部分company 例外處理
corp_name[324] = 'News Class A'
corp_name[325] = 'News Class B'
corp_name[198] = 'Gap Inc.'
# print(corp_name[2], symbol_data.iloc[2,0])

In [ ]:
# 剔除意義模糊的公司代碼
symbol_blacklist = ['MMM', 'ARE', 'AES', 'ALL', 'ABC', 'BLL', 'CAH', 'CAT',\
                   'CSX', 'CME', 'COST', 'CVS', 'EBAY', 'FMC', 'GIS', 'HES',\
                   'ICE', 'KEY', 'LIN', 'LEN', 'LOW', 'MET', 'ROST', 'LUV',\
                   'TEL', 'TXT', 'COO', 'MOS', 'DIS', 'USB', 'BIO','HAS',
                   'NOW','WELL','POOL','FAST','SEE']

1.蒐集CNN新聞資料

In [ ]:
def search(data_list):
    page = 1
    while True:
        time.sleep(random.randint(2,3))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        corp_arti = soup.find_all(class_= "cnn-search__result--article")

        for arti in corp_arti:
            data.append({
                'headline': arti.find(class_ = 'cnn-search__result-headline').text.replace('\n', ''),
                'date'    : arti.find(class_ = 'cnn-search__result-publish-date').text.replace('\n', ''),
                'link'    : arti.find('a')['href'].replace('//', ''),
                'text'    : arti.find(class_ = 'cnn-search__result-body').text.replace('\n', '').replace("  ", ''),
            })
        print('page: ', page, " total_data: ", len(data))    
        #換頁(Next)
        try:
            #如果Next存在
            e = driver.find_element_by_css_selector('body > div.pg-search.pg-wrapper > div.pg-no-rail.pg-wrapper > div > div.l-container > div.cnn-search__right > div > div.cnn-search__results-pagi > div > div.pagination-arrow.pagination-arrow-right.cnnSearchPageLink.text-active')
            e.click()

            if page == 1000:#頁數超過1000頁，無法抓取多出的data
                break
        except:
            break
        page += 1
    return data_list

In [ ]:
# CNN 網站的新聞類別
category = ['us,politics,world,opinion,health', 'entertainment', 'sport', 'travel', 'style']

In [ ]:
from selenium.webdriver.firefox.options import Options

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options,executable_path='/usr/bin/geckodriver')

In [ ]:
%%time
for i in range(4,len(category)) :
    category_name = category[i]
    print("********" , category_name, "***********")
    num  = 0
    for k  in range(0,len(corp_name)):
        if num == 5: # 避免網頁當掉
            driver.quit()
            options.headless = True
            driver = webdriver.Firefox(options=options,executable_path='/usr/bin/geckodriver')

        # 根據Company Name蒐集資料
        print("----",k, corp_name[k],"----")
        name_qoute = quote(corp_name[k], safe='')
        url = 'https://edition.cnn.com/search?q='+ name_qoute +'&size=10&type=article&'\
        +'category=' + category_name
        driver.get(url)

        data = []

        data = search(data) #def

        # 根據Symbol搜集資料
        if (symbol_data.iloc[k,0] not in symbol_blacklist) and (len(symbol_data.iloc[k,0]) > 2):
            name_qoute = quote(symbol_data.iloc[k,0], safe='')
            url = 'https://edition.cnn.com/search?q='+ name_qoute +'&size=10&type=article&'\
            +'category=' + category_name
            driver.get(url)
            search(data) #def

        ## 判別是否有資料，有的話存成csv，沒有的話另外紀錄名稱為txt，以便核對
        if len(data) != 0: 
            links_data = pd.DataFrame(data)
            links_data = links_data.drop_duplicates(subset=['link'])
            print("data_len", len(data))
            out = os.path.join(category_name , symbol_data.iloc[k,0]+'.csv')
            links_data.to_csv(out, index = False)
        else:
            print(symbol_data.iloc[k,0], 'No data')
            with open(os.path.join(category_name+'_NoData_file.txt'), 'a') as f:
                f.write( symbol_data.iloc[k,0] + "\n" )
        num += 1
#     break
driver.quit()

2.量化新聞資料

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pysentiment2 as ps
import string
import xlrd
import openpyxl

處理token

In [ ]:
stop_words = stopwords.words('english')
exclude = set(string.punctuation)

#忽略隱藏檔
files = [i for i in os.listdir(folder_path) if not i.startswith('.')]

for i in range(len(files)):
    data_read = os.path.join(folder_path, files[i] )
    comment_data = pd.read_csv(data_read)
    print(i, data_read)
    
    comment_data['headline_token'] = None
    comment_data['text_token']  = None
    
    # 計算共有幾則評論
    review_num = 0 

    # title
    for j in range(0, len(comment_data)):
        content = str(comment_data['headline'][j]).lower()
        # 移除標點符號
        text = ''.join(ch for ch in content if ch not in exclude)
        # tokeing 
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        comment_data.iloc[j, -2] = ','.join(x for x in tokens)

    # content
    for k in range(0, len(comment_data)):
        content = str(comment_data['text'][k]).lower()
        # 移除標點符號
        text = ''.join(ch for ch in content if ch not in exclude)
        # tokeing 
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        comment_data.iloc[k, -1] = ','.join(x for x in tokens)
        
        review_num += 1
    csv = files[i].replace('.csv', '_token.csv')
    output = os.path.join(os.getcwd(),'sanfish','Business_token', csv)
    comment_data.to_csv(output, index = True)
    print("reviews: ", review_num)
    print("-------")

計算情緒詞彙

In [ ]:
hiv4 = ps.HIV4()

In [ ]:
files = [i for i in os.listdir(folder_path) if not i.startswith('.')]
for i in range(len(files)):
    data_read = os.path.join(folder_path, files[i])
    token_data = pd.read_csv(data_read, index_col=0)
    print('**', i , files[i], 'len:', len(token_data), '**')
    
    token_data["headline_pos"]      = None #6
    token_data["headline_neg"]      = None #7
    token_data["headline_netural"]  = None #8
    token_data["headline_total"]    = None #9
    
    token_data["text_pos"]      = None #10
    token_data["text_neg"]      = None #11
    token_data["text_netural"]  = None #12
    token_data["text_total"]    = None #13
    
    for j in range(0, token_data.shape[0]):
        if(j%100==0):
            print(j)
    # title
        title_pos = 0
        title_neg = 0
        title_netural = 0
        title_total   = 0
        if type(token_data["headline_token"][j]) != float:
            for title_word in token_data["headline_token"][j].split(","):
                tokens = hiv4.tokenize(title_word)
                score = hiv4.get_score(tokens)
                
                # 分類詞性
                if(score["Polarity"] > 0):
                    title_pos += 1
                elif (score["Polarity"] < 0) :
                    title_neg += 1
                else:
                    title_netural += 1
                token_data.iloc[j, 6] = title_pos
                token_data.iloc[j, 7] = title_neg
                token_data.iloc[j, 8] = title_netural
                token_data.iloc[j, 9] = len(token_data["headline_token"][j].split(","))
        # text
        text_pos = 0
        text_neg = 0
        text_netural = 0
        text_total   = 0
        if type(token_data["text_token"][j]) != float:
            for word in token_data["text_token"][j].split(","):
                tokens = hiv4.tokenize(word)
                score = hiv4.get_score(tokens)
                if(score["Polarity"] > 0):
                    text_pos += 1
                elif (score["Polarity"] < 0) :
                    text_neg += 1
                else:
                    text_netural += 1
                token_data.iloc[j, -4] = text_pos
                token_data.iloc[j, -3] = text_neg
                token_data.iloc[j, -2] = text_netural
                token_data.iloc[j, -1] = len(token_data["text_token"][j].split(","))
    csv = files[i].replace('_token.csv', '_analyzed.csv')
    output = os.path.join(os.getcwd(),'Business2','Business_analyzed',csv)
    token_data.to_csv(output, index=False)

In [ ]:
from collections import Counter

In [ ]:
read = os.path.join(os.getcwd(),'SP500_merge.xlsx')
symbol_data = pd.read_excel(read, index_col = 0 )
symbol = symbol_data['Company name'].to_list()


corp_name = []
for i, element in enumerate(list(symbol)):
    ele_low = element.lower()
    ele_low = ele_low.replace(" company", '').replace(' corp', '').replace(' inc', '')\
    .replace(" plc", '').replace(' class a', '').replace(' class c', '')\
    .replace(" class b", '').replace(" ltd.", "").replace(".com", "")\
    .replace(" int'l", "").replace(" corporation", "").replace(" group", "")\
    .replace(" svc.gp.", "").replace(" incorporated", "")
    corp_name.append(ele_low)

# 部分company
corp_name[324] = 'News Class A'
corp_name[325] = 'News Class B'
corp_name[198] = 'Gap Inc.'
print(corp_name[3], symbol_data.iloc[3,0])

In [ ]:
symbol_blacklist = ['MMM', 'ARE', 'AES', 'ALL', 'ABC', 'BLL', 'CAH', 'CAT',\
                   'CSX', 'CME', 'COST', 'CVS', 'EBAY', 'FMC', 'GIS', 'HES',\
                   'ICE', 'KEY', 'LIN', 'LEN', 'LOW', 'MET', 'ROST', 'LUV',\
                   'TEL', 'TXT', 'COO', 'MOS', 'DIS', 'USB', 'BIO','HAS',
                   'NOW','WELL','POOL','FAST','SEE']

In [ ]:
folder_path = os.path.join(os.getcwd(),'travel1','travel_analyzed')
files = [i for i in os.listdir(folder_path) if not i.startswith('.')]
len(files)

3.過濾新聞內文

In [ ]:
folder_path = os.path.join(os.getcwd(),'travel1','travel_analyzed')
files = [i for i in os.listdir(folder_path) if not i.startswith('.')]
len(files)

In [ ]:
for j in range(len(files)):
    f = files[j]
    sym = f.replace('_analyzed.csv','')
    i = symbol_data[symbol_data["Symbol"]==sym].index[0]
    print(j,'index', i ,sym)
    data_read = os.path.join(folder_path, f)
    token_data = pd.read_csv(data_read)
    token_data = token_data.dropna(subset = ['text']) # 確保text沒有空值
    
    lower = token_data['text'].str.lower()
    c1 = lower.str.contains(corp_name[i]) #確認公司名稱是否在本文裡    
    
    if len(sym) < 3 or sym in symbol_blacklist: #如果len(symbol)<3 or 在沒有搜尋的名單中
        token_data2 = token_data[ c1 ]
        print('  ***', 'c1', Counter(lower.str.contains(corp_name[i])), 'len2:', len(token_data2))

    else:
        c2 = token_data['text'].str.contains(symbol_data.iloc[i,0])
        c = [a or b for a, b in zip(c1, c2)]
        token_data2 = token_data[c]
        print('  ***','len1:', len(token_data),'len2:', len(token_data2))
    
    csv = f.replace('_token.csv', '_analyzed.csv')
    output = os.path.join(os.getcwd(),'travel2','travel_analyzed', csv)
    token_data2.to_csv(output, index= False)
#     break

In [ ]:
for category in category_list:
    news_path = os.path.join(os.getcwd(), category+'2', category+'_analyzed')
    print(news_path)

In [ ]:
# 將每家公司不同類別的新聞合併成一個檔案

symbol_dic = defaultdict()
for category in category_list:
    print(category)
    
    news_path = os.path.join(os.getcwd(), category+'2', category+'_analyzed')
#     print(news_path)
    news_files = [i for i in os.listdir(news_path) if not i.startswith('.')]
    
    for j in range(len(symbol_list)):
        symbol = symbol_list[j]# 公司代碼
#         print(symbol)
        try:
            if symbol+'_analyzed.csv' in news_files:
                
                read_path = os.path.join(news_path, symbol+'_analyzed.csv') # 讀取data的路徑
                news_data = pd.read_csv(read_path)

                if symbol in symbol_dic.keys():
                    symbol_dic[symbol] = pd.concat([symbol_dic[symbol], news_data] ) 
                else:
                    symbol_dic[symbol] = news_data
        
        except NotADirectoryError:
            continue

In [ ]:
#根據key 將每間公司存成獨立檔案
for key, value in symbol_dic.items():
    print(key)
    dateFormatter = "%b-%d-%Y"
    value['date'] = value['date'].apply(lambda x: x.replace(' ', '-').replace(',',''))
    value['date'] = value['date'].apply(lambda x: datetime.strptime(x, dateFormatter).date())
    csv = key + '_analyzed.csv'
    out_path = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory_analyzed', csv)
    value.to_csv( out_path, index = False )

4.合併日期＆計算情緒指數

In [ ]:
for i in range(0,len(files)):
    data_read = os.path.join(folder_path, files[i])
    merged_data = pd.read_csv(data_read)
    
    merged_data["amount"] = 1
    aggregation_functions = {
            'headline_pos': 'sum', 'headline_neg': "sum",'headline_netural': 'sum', 'headline_total': 'sum',\
            'text_pos': 'sum', 'text_neg': "sum",'text_netural': 'sum', 'text_total': 'sum',\
            'amount': "sum" }
    merged_data = merged_data.groupby(merged_data['date']).aggregate(aggregation_functions)
    merged_data = merged_data.reset_index()
    
    merged_data['headline_Pos_ratio']     = None #10
    merged_data['headline_Neg_ratio']     = None #11
    merged_data['headline_Netural_ratio'] = None #12
    merged_data['headline_Polarity']      = None #13
    merged_data['headline_Subjectivity']  = None #14
    
    merged_data['text_Pos_ratio']         = None #15
    merged_data['text_Neg_ratio']         = None #16
    merged_data['text_Netural_ratio']     = None #17
    merged_data['text_Polarity']          = None #18
    merged_data['text_Subjectivity']      = None #19
    
    for w in range(0, merged_data.shape[0]):
        if(merged_data["headline_pos"].iloc[w] + merged_data["headline_neg"].iloc[w]!=0):#分母不可為0
            merged_data.iloc[w, 10] = merged_data["headline_pos"].iloc[w]\
            /(merged_data["headline_pos"].iloc[w]+merged_data["headline_neg"].iloc[w])#head_Pos_ratio

            merged_data.iloc[w, 11] = merged_data["headline_neg"].iloc[w]\
            /(merged_data["headline_pos"].iloc[w]+merged_data["headline_neg"].iloc[w])#head_Neg_ratio

            merged_data.iloc[w, 13] = (merged_data["headline_pos"].iloc[w]-merged_data["headline_neg"].iloc[w])\
            /(merged_data["headline_pos"].iloc[w]+merged_data["headline_neg"].iloc[w])#head_Polarity
        else:
            merged_data.iloc[w, 10] = 0 #head_Pos_ratio
            merged_data.iloc[w, 11] = 0 #head_Neg_ratio        
            merged_data.iloc[w, 13] = 0 #head_Polarity

        if(merged_data["headline_total"].iloc[w]!=0):#分母不可為0
            merged_data.iloc[w, 12] = merged_data["headline_netural"].iloc[w]/merged_data["headline_total"].iloc[w]#head_Netural_ratio

            merged_data.iloc[w, 14] = (merged_data["headline_pos"].iloc[w]+merged_data["headline_neg"].iloc[w])/ \
            (merged_data["headline_total"].iloc[w])#head_Subjectivity
        else:
            merged_data.iloc[w, 12] = 0 #head_Netural_ratio
            merged_data.iloc[w, 14] = 0 #head_Subjectivity


        if(merged_data["text_pos"].iloc[w]+merged_data["text_neg"].iloc[w]!=0):#分母不可為0
            merged_data.iloc[w, 15] = merged_data["text_pos"].iloc[w]\
            /(merged_data["text_pos"].iloc[w]+merged_data["text_neg"].iloc[w])#text_Pos_ratio

            merged_data.iloc[w, 16] = merged_data["text_neg"].iloc[w]\
            /(merged_data["text_pos"].iloc[w]+merged_data["text_neg"].iloc[w])#text_Neg_ratio

            merged_data.iloc[w, 18] = (merged_data["text_pos"].iloc[w]-merged_data["text_neg"].iloc[w])\
            /(merged_data["text_pos"].iloc[w]+merged_data["text_neg"].iloc[w])#text_Polarity     
        else:
            merged_data.iloc[w, 15] = 0 #text_Pos_ratio
            merged_data.iloc[w, 16] = 0 #text_Neg_ratio
            merged_data.iloc[w, 18] = 0 #text_Polarity

        if(merged_data["text_total"].iloc[w] != 0):#分母不可為0
            merged_data.iloc[w, 17] =  merged_data["text_netural"].iloc[w]/merged_data["text_total"].iloc[w]#text_Netural_ratio
            merged_data.iloc[w, 19] = (merged_data["text_pos"].iloc[w]+merged_data["text_neg"].iloc[w])\
            /merged_data["text_total"].iloc[w]#text_Subjectivity
        else:
            merged_data.iloc[w, 17] = 0 #text_Netural_ratio
            merged_data.iloc[w, 19] = 0 #text_Subjectivity    
            
    csv = files[i].replace('_analyzed.csv', '_merged.csv')
    output = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory_merged',csv)
    merged_data.to_csv(output, index=False)
    print(i, csv)

5.標準化情緒指標

In [ ]:
def standsrd(x):
    return (x - x.mean(skipna=True))/ x.std(skipna=True)

In [ ]:
path = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory_merged')
files = [i for i in os.listdir(path) if not i.startswith('.')]
len(files)

In [ ]:
for i in range(len(files)):
    
    data_read = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory_merged', files[i])
    data = pd.read_csv(data_read)
    
    data["headline_sPos_ratio"]     = None # 20
    data["headline_sNeg_ratio"]     = None # 21
    data["headline_sNetural_ratio"] = None # 22
    data["headline_sPolarity"]      = None # 23
    data["headline_sSubjectivity"]  = None # 24

    data["text_sPos_ratio"]         = None # 25
    data["text_sNeg_ratio"]         = None # 26
    data["text_sNetural_ratio"]     = None # 27
    data["text_sPolarity"]          = None # 28
    data["text_sSubjectivity"]      = None # 29

    data['headline_sPos_ratio']     = standsrd(data["headline_Pos_ratio"])
    data['headline_sNeg_ratio']     = standsrd(data["headline_Neg_ratio"])
    data['headline_sNetural_ratio'] = standsrd(data["headline_Netural_ratio"])
    data['headline_sPolarity']      = standsrd(data["headline_Polarity"])
    data['headline_sSubjectivity']  = standsrd(data["headline_Subjectivity"])


    data['text_sPos_ratio']     = standsrd(data["text_Pos_ratio"])
    data['text_sNeg_ratio']     = standsrd(data["text_Neg_ratio"])
    data['text_sNetural_ratio'] = standsrd(data["text_Netural_ratio"])
    data['text_sPolarity']      = standsrd(data["text_Polarity"])
    data['text_sSubjectivity']  = standsrd(data["text_Subjectivity"])
    
    csv = files[i].replace('_merged.csv', '.csv')
    out_path = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory', csv)
    data.to_csv( out_path, index = False )
    print('***',i,csv,'***')

6.處理Control variable

In [ ]:
# 用CNN有的company_symbol 作為 symbol
read = os.path.join(os.getcwd(), 'CNN_NoCategory2', 'CNN_NoCategory')
CNN_files = [i for i in os.listdir(read) if not i.startswith('.')]
len(CNN_files)

In [ ]:
# daily path
daily_path = os.path.join(os.getcwd(),'daily_0628')
daily_files = [i for i in os.listdir(daily_path) if not i.startswith('.')]
len(daily_files)

In [ ]:
# Market_cap path
cap_path = os.path.join(os.getcwd(),'S&P500_marketcap_0622')
cap_files = [i for i in os.listdir(cap_path) if not i.startswith('.')]
len(cap_files)

In [ ]:
for i in range(len(CNN_files)):
    symbol = CNN_files[i]
    print(i, CNN_files[i])
    
    ### daily處理
    daily_read = os.path.join(daily_path, symbol) # 讀取對應公司的daily資料
    daily_data = pd.read_csv(daily_read)

    daily_data['return']     = None # 6
    daily_data['adj_return'] = None
    daily_data['volume']     = None
    daily_data['volatility'] = None
    daily_data['spread']     = None

    # 計算 return & adj_close
    daily_data['return'] = daily_data['Close'] / daily_data['Close'].shift(1)
    daily_data['return'] = daily_data['return'].apply(lambda x: math.log(x))
    
    daily_data['adj_return'] = daily_data['Adj Close'] / daily_data['Adj Close'].shift(1)
    daily_data['adj_return'] = daily_data['adj_return'].apply(lambda x: math.log(x))
    
    # 計算 volatility & spread 
    High  = daily_data['High'].apply(lambda x: math.log(x))
    Open  = daily_data['Open'].apply(lambda x: math.log(x))
    Close = daily_data['Close'].apply(lambda x: math.log(x))
    Low   = daily_data['Low'].apply(lambda x: math.log(x))

    daily_data['volatility'] = (High - Close)*(High - Open) + (Low - Close) * (Low - Open)
    daily_data['spread'] = 4*( Close - ( Low+High )/2 )*( Close - ( Low.shift(-1)+High.shift(-1) ) /2)
    daily_data['spread'] = daily_data['spread'].apply(lambda x: max(x,0)**(1/2))

    ### capitial 處理
    cap_read = os.path.join(cap_path, CNN_files[i]) # 讀取對應公司的市值資料
    cap_data = pd.read_csv(cap_read)
    # 將cap取log e
    cap_data['Market_Cap'] = cap_data['Market_Cap'].apply(lambda x: math.log((x/1000000)))

    ### merge data：根據market_cap的日期做merge，再移除沒有return的column
    feature_data = pd.merge(daily_data, cap_data, how="right", on="Date")
    feature_data['volume'] = (feature_data['Volume'] - feature_data['Volume'].mean())\
                            / feature_data['Volume'].mean() # 根據有cap的日期計算volume的mean
    feature_data = feature_data.dropna() # 沒有做reindex，因為最後不會存入index
#     feature_data = feature_data.dropna(subset=['return'])
    
    feature_data = feature_data.rename(columns = {'Market_Cap':'size'})

    ### save
    out_path = os.path.join(os.getcwd(), 'daily_CNN_feature3', CNN_files[i])
    feature_data.to_csv(out_path, index= False)

7.合併新聞＆股價資料

In [ ]:
# daily path
daily_path = os.path.join(os.getcwd(),'daily_CNN_feature3')
daily_files = [i for i in os.listdir(daily_path) if not i.startswith('.')]
len(daily_files)

In [ ]:
# news path
news_path = os.path.join(os.getcwd(),'CNN_NoCategory2','CNN_NoCategory')
news_files = [i for i in os.listdir(daily_path) if not i.startswith('.')]
len(news_files)

In [ ]:
df = []
news_feature = ['headline_sPos_ratio','headline_sNeg_ratio','headline_sNetural_ratio','headline_sPolarity',\
                'headline_sSubjectivity','text_sPos_ratio','text_sNeg_ratio','text_sNetural_ratio',\
                'text_sPolarity','text_sSubjectivity'] # 再merge之後需要fillna的值，先存成list
use_num = 0
for i in range(len(daily_files)):
# for i in range(2):
    print(daily_files[i])
    ### daily處理
    daily_read = os.path.join(daily_path, daily_files[i])
    daily_data = pd.read_csv(daily_read)
    daily_data = daily_data.drop(daily_data.loc[:, 'Open':'Volume'].columns, axis =1)
    
    ### news 處理
    news_read = os.path.join(news_path, daily_files[i])
    news_data = pd.read_csv(news_read)
    news_data = news_data.rename(columns = {'date':'Date'}) # 便於合併到daily_feature 
    news_data = news_data.drop(news_data.loc[:, 'headline_pos':'text_Subjectivity'].columns, axis =1) #砍掉不必要的columns
        
    
    ### merge daily & news：先不將news做t-1的動作，直接與return的日期做對應，接著直接shift
    feature_data = pd.merge(daily_data, news_data, how="left", on="Date")
    use_num += feature_data['text_sPos_ratio'].notna().sum() # 計算最後所用到的新聞數量
    feature_data[news_feature] = feature_data[news_feature].fillna(value=0) #將news_feature中nan值補0
    
    shift_list = feature_data.loc[:, 'return':'text_sSubjectivity'].columns.tolist() # 需要sfift的columns做成list
    for j in range(1,6):
        for name in shift_list:
            feature_data[name+ '_lag' + str(j)] = feature_data[ name ].shift(j)
            
    feature_data = feature_data.drop(feature_data.loc[:, 'volume':'text_sSubjectivity'].columns, axis =1) #刪掉不必要的欄位
    feature_data = feature_data.iloc[5:] # 砍掉前五筆經過shift後有nan值得row
    feature_data.reset_index(inplace = True)
    feature_data = feature_data.drop(columns = ["index"] )
    
    ### 每家公司處理完先獨立存在folder
    feature_data.insert(0, 'company', '') # 填入公司代號
    feature_data['company'] = daily_files[i].replace('.csv','')
    feature_data.to_csv(os.path.join(os.getcwd(), 'Feature_CNN_Nocate3', daily_files[i] ), index= False)
    
    ### 將每家公司concat
    if len(df) != 0:
        df = pd.concat([df, feature_data])
    else:
        df = pd.concat([feature_data])
df.to_csv(os.path.join(os.getcwd(), 'CNN_Nocate_Regression3.csv'), index= False)

8.Regression

In [ ]:
import pandas as pd
from linearmodels.panel import RandomEffects
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare
import statsmodels.api as sm
import os
from datetime import datetime

In [ ]:
data = pd.read_csv( os.path.join(os.getcwd(), 'CNN_Nocate_Regression3.csv' ) )
data['Date'] = data['Date'].apply( lambda x: datetime.strptime(x, '%Y-%m-%d').date() )
data = data.set_index(["company", "Date"])

In [ ]:
x_var = ['news_variable',\
        'return_lag1','volume_lag1','volatility_lag1','spread_lag1','size_lag1',\
        'return_lag2','volume_lag2','volatility_lag2','spread_lag2','size_lag2',\
        'return_lag3','volume_lag3','volatility_lag3','spread_lag3','size_lag3',\
        'return_lag4','volume_lag4','volatility_lag4','spread_lag4','size_lag4',\
        'return_lag5','volume_lag5','volatility_lag5','spread_lag5','size_lag5']  

In [ ]:
head = ['headline_sPos_ratio_lag', 'headline_sNeg_ratio_lag', 'headline_sNetural_ratio_lag', 'headline_sPolarity_lag', 'headline_sSubjectivity_lag']
text = ['text_sPos_ratio_lag', 'text_sNeg_ratio_lag', 'text_sNetural_ratio_lag', 'text_sPolarity_lag', 'text_sSubjectivity_lag']
news_var = [head, text]
for name in news_var:
    for i in range(len(name)):
        x = name[i]+str(1)
        x_var[0] = x
        print('*** news_var:',x ,'len:',len(x_var))
        
        y = data['adj_return']
        exog_vars = x_var
        exog = sm.add_constant(data[exog_vars])
        mod = PanelOLS(y, exog,  time_effects = True, entity_effects = True )

        if i % 5 == 0:
            # pos = x
            pos = mod.fit()
        elif i % 5 == 1:
            # neg = x
            neg = mod.fit()
        elif i % 5 == 2:
            # netural = x
            netural = mod.fit()
        elif i % 5 == 3:
            # polarity = x
            polarity = mod.fit()
        else:
            # subjectivity = x
            subjectivity = mod.fit()

    # print(pos, neg, netural, polarity, subjectivity)
    output = compare({"Pos": pos, "Neg": neg, "Netural":netural, "Polarity": polarity, "Subjectivity": subjectivity},precision= 'tstats', stars=True)
    result = output.summary.as_csv()
    result = result.replace("Model Comparison","")
    result = ",".join(result.split("=")[:-1])
    result = result.replace(" ", "")
    result = [j for j in result.split(",") if j != ""]
    result = ",".join(result)
    result = result.replace("\n,\n", "\n")

    with open (os.path.join(os.getcwd(),'cnn_singlePanelOLS_adj.csv'), 'a') as file:
        file.write(result)
    
    # break

In [ ]:
news_var = ['headline_sPos_ratio_lag', 'headline_sNeg_ratio_lag', 'headline_sNetural_ratio_lag', 'headline_sPolarity_lag', 'headline_sSubjectivity_lag',\
            'text_sPos_ratio_lag', 'text_sNeg_ratio_lag', 'text_sNetural_ratio_lag', 'text_sPolarity_lag', 'text_sSubjectivity_lag']

multi_var = ['one', 'two', 'three', 'four', 'five',\
        'adj_return_lag1','volume_lag1','volatility_lag1','spread_lag1','size_lag1',\
        'adj_return_lag2','volume_lag2','volatility_lag2','spread_lag2','size_lag2',\
        'adj_return_lag3','volume_lag3','volatility_lag3','spread_lag3','size_lag3',\
        'adj_return_lag4','volume_lag4','volatility_lag4','spread_lag4','size_lag4',\
        'adj_return_lag5','volume_lag5','volatility_lag5','spread_lag5','size_lag5']  

In [ ]:
### one panel: multipule news variable
for var in news_var:
    # print(var)
    for i in range(1,6):
        x = var + str(i)
        multi_var[i-1] = x
    print('*** muti_var:',x ,'len:',len(multi_var))
    
    y = data['adj_return']
    exog_vars = multi_var
    exog = sm.add_constant(data[exog_vars])

    ### PanelOLS: time_effects
    mod = PanelOLS(y, exog, time_effects = True, entity_effects = True)
    fixedPanel = mod.fit()
    
    ### compare all panel
    output = compare({"PanelOLS": fixedPanel}, precision='tstats', stars=True)
    # print(output)
    result = output.summary.as_csv()
    result = result.replace("Model Comparison","")
    result = ",".join(result.split("=")[:-1])
    result = result.replace(" ", "")
    result = [j for j in result.split(",") if j != ""]
    result = ",".join(result)
    result = result.replace("\n,\n", "\n")

    with open (os.path.join(os.getcwd(),'cnn_multiPanelOLS_adj.csv'), 'a') as file:
        file.write(result)

    # break